# 🗺️ Traffic Speed Visualization (MapboxGL)

In [ ]:
# 📦 Install dependencies (if not already installed)
!pip install requests pandas geopandas mapboxgl shapely --quiet

In [ ]:
# 🔧 Setup
import requests
import pandas as pd
from mapboxgl.viz import ChoroplethViz
from mapboxgl.utils import create_color_stops

# Replace with your actual Mapbox token
MAPBOX_TOKEN = "your_mapbox_token_here"

# API base URL
BASE_URL = "http://localhost:8000"

In [ ]:
# 🧪 Query aggregated API endpoint
params = {
    "day": "Monday",
    "period": "AM Peak"
}
response = requests.get(f"{BASE_URL}/aggregates/", params=params)

# GeoJSON-style features
geojson_data = response.json()
print(f"Retrieved {len(geojson_data)} features")

In [ ]:
# 🎨 Build color-coded map
features = [
    {
        "type": "Feature",
        "geometry": f["geometry"],
        "properties": {
            "average_speed": f["properties"]["average_speed"],
            "road_name": f["properties"].get("road_name", "Unnamed")
        }
    }
    for f in geojson_data
]

viz = ChoroplethViz(
    {
        "type": "FeatureCollection",
        "features": features
    },
    access_token=MAPBOX_TOKEN,
    color_property="average_speed",
    color_stops=create_color_stops([10, 20, 30, 40, 50], colors='Reds'),
    center=(-81.6557, 30.3322),
    zoom=11,
    line_width=1.5,
    opacity=0.8,
    legend_title="Average Speed (mph)"
)
viz.show()

In [ ]:
# 📊 Optional Tabular Summary
df = pd.DataFrame([
    {
        "road_name": f["properties"].get("road_name", "Unnamed"),
        "avg_speed": f["properties"]["average_speed"]
    }
    for f in geojson_data
])

df.sort_values("avg_speed").head(10)